# 📊 Dashboard Básico con Streamlit

## Objetivo
Crear un dashboard interactivo usando Python y Streamlit como alternativa a herramientas no-code como Looker Studio.

## ¿Por qué Streamlit?
- **100% Python**: Usa el lenguaje que ya conoces
- **Interactivo**: Crea filtros y controles con pocas líneas
- **Gratuito**: Open source, hosting gratuito en Streamlit Cloud
- **Rápido**: De código a dashboard en minutos

## Casos de uso
- Cuando necesitas personalización avanzada
- Cuando tus datos requieren procesamiento con Python
- Cuando quieres control total del diseño
- Cuando trabajas en equipos técnicos

## 📐 Principios de Diseño de Dashboards

Antes de escribir código, necesitas entender cómo las personas leen dashboards:

### 1. **Jerarquía Visual (Arriba → Abajo)**
```
┌─────────────────────────────────────┐
│  KPIs CRÍTICOS (más importantes)    │ ← ARRIBA
├─────────────────────────────────────┤
│  Gráficas de Tendencias             │ ← MEDIO
├─────────────────────────────────────┤
│  Tablas de Detalle                  │ ← ABAJO
└─────────────────────────────────────┘
```

### 2. **Flujo de Lectura (Izquierda → Derecha)**
```
┌─────────────┬─────────────┬─────────────┐
│   KPI #1    │   KPI #2    │   KPI #3    │
│ Satisfacción│  Atendidos  │   Tiempo    │
└─────────────┴─────────────┴─────────────┘
```

### 3. **General → Particular**
- Empieza con métricas agregadas (promedio de satisfacción)
- Sigue con comparaciones (satisfacción por área)
- Termina con detalles (tabla de beneficiarios individuales)

### 4. **Filtros en Sidebar (Izquierda)**
- Los controles siempre van en la barra lateral
- Esto permite que el contenido principal sea el foco

## 🛠️ Instalación

Si no tienes Streamlit instalado, ejecuta:

```bash
pip install streamlit pandas plotly
```

## 📝 Paso 1: Crear el Archivo Principal

**IMPORTANTE**: Streamlit NO funciona directamente en Jupyter notebooks. Necesitas:

1. Crear un archivo `.py` con el código
2. Ejecutar `streamlit run archivo.py` desde la terminal

Este notebook te guía en cómo escribir ese archivo paso a paso.

## 📦 Paso 2: Imports y Configuración

Copia este código en un archivo llamado `dashboard_ong.py`:

In [ ]:
# dashboard_ong.py
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Configuración de la página
st.set_page_config(
    page_title="Dashboard ONG",
    page_icon="📊",
    layout="wide",  # Usa todo el ancho de la pantalla
    initial_sidebar_state="expanded"  # Sidebar visible al inicio
)

### 🔍 Explicación:
- `layout="wide"`: Aprovecha todo el ancho de pantalla (importante para dashboards)
- `page_icon`: Aparece en la pestaña del navegador
- `initial_sidebar_state`: Sidebar abierto por defecto

## 📂 Paso 3: Cargar y Cachear Datos

Agrega esta función al archivo:

In [ ]:
@st.cache_data  # Cachea los datos para no recargarlos en cada interacción
def cargar_datos():
    """
    Carga el CSV de datos de la ONG.
    El cache hace que solo se cargue una vez, mejorando performance.
    """
    df = pd.read_csv('../datos/datos_ong_ejemplo.csv')
    
    # Convertir fecha a datetime
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Crear columna de mes para análisis temporal
    df['mes'] = df['fecha'].dt.to_period('M').astype(str)
    
    return df

# Cargar datos
df = cargar_datos()

### 🔍 Explicación:
- `@st.cache_data`: Decorator que guarda el resultado en cache
- Sin cache, cada vez que cambias un filtro se recargaría el CSV completo
- Esto es CRÍTICO para performance en dashboards grandes

## 🎨 Paso 4: Título y Descripción

El título va arriba de todo, como en cualquier documento:

In [ ]:
# Título principal
st.title('📊 Dashboard de Atención a Beneficiarios')
st.markdown('### Análisis de Satisfacción y Operaciones')
st.markdown('---')  # Línea divisoria

## 🔧 Paso 5: SIDEBAR - Filtros Interactivos (IZQUIERDA)

**PRINCIPIO**: Los controles siempre van en el sidebar (izquierda). Esto deja el área principal para visualizaciones.

In [ ]:
# SIDEBAR: Filtros
st.sidebar.header('🔍 Filtros')

# Filtro 1: Área geográfica
areas_disponibles = ['Todas'] + sorted(df['area'].unique().tolist())
area_seleccionada = st.sidebar.selectbox(
    'Área Geográfica:',
    areas_disponibles
)

# Filtro 2: Programa
programas_disponibles = ['Todos'] + sorted(df['programa'].unique().tolist())
programa_seleccionado = st.sidebar.selectbox(
    'Programa:',
    programas_disponibles
)

# Filtro 3: Rango de satisfacción
satisfaccion_min, satisfaccion_max = st.sidebar.slider(
    'Rango de Satisfacción:',
    min_value=int(df['satisfaccion'].min()),
    max_value=int(df['satisfaccion'].max()),
    value=(int(df['satisfaccion'].min()), int(df['satisfaccion'].max()))
)

st.sidebar.markdown('---')
st.sidebar.markdown('**💡 Tip**: Cambia los filtros para explorar los datos')

### 🔍 Explicación:
- `st.sidebar.*`: Todo lo que va en sidebar usa este prefijo
- `selectbox`: Dropdown para seleccionar una opción
- `slider`: Control deslizante para rangos numéricos
- Agregamos 'Todas' y 'Todos' para permitir ver todo sin filtrar

## 🎯 Paso 6: Aplicar Filtros a los Datos

In [ ]:
# Aplicar filtros
df_filtrado = df.copy()

if area_seleccionada != 'Todas':
    df_filtrado = df_filtrado[df_filtrado['area'] == area_seleccionada]

if programa_seleccionado != 'Todos':
    df_filtrado = df_filtrado[df_filtrado['programa'] == programa_seleccionado]

df_filtrado = df_filtrado[
    (df_filtrado['satisfaccion'] >= satisfaccion_min) &
    (df_filtrado['satisfaccion'] <= satisfaccion_max)
]

# Mostrar cuántos registros quedaron después del filtro
st.info(f'📊 Mostrando **{len(df_filtrado)}** registros de **{len(df)}** totales')

## 📊 Paso 7: KPIs en la PARTE SUPERIOR (Nivel 1 - Más Importante)

**PRINCIPIO**: Los KPIs van arriba porque son lo primero que debe ver el usuario. Son el resumen ejecutivo.

In [ ]:
# Calcular KPIs
satisfaccion_promedio = df_filtrado['satisfaccion'].mean()
total_beneficiarios = len(df_filtrado)
tiempo_promedio = df_filtrado['tiempo_atencion_min'].mean()
tasa_resolucion = (df_filtrado['resolucion_exitosa'] == 'SI').sum() / len(df_filtrado) * 100

# Mostrar KPIs en 4 columnas (izquierda → derecha)
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        label="😊 Satisfacción Promedio",
        value=f"{satisfaccion_promedio:.1f}/10",
        delta=f"{satisfaccion_promedio - df['satisfaccion'].mean():.1f} vs. total"
    )

with col2:
    st.metric(
        label="👥 Total Beneficiarios",
        value=f"{total_beneficiarios:,}",
        delta=None
    )

with col3:
    st.metric(
        label="⏱️ Tiempo Promedio",
        value=f"{tiempo_promedio:.0f} min",
        delta=f"{tiempo_promedio - df['tiempo_atencion_min'].mean():.0f} vs. total",
        delta_color="inverse"  # Menos tiempo es mejor
    )

with col4:
    st.metric(
        label="✅ Tasa de Resolución",
        value=f"{tasa_resolucion:.1f}%",
        delta=None
    )

st.markdown('---')

### 🔍 Explicación:
- `st.columns(4)`: Divide el espacio en 4 columnas iguales
- `st.metric`: Widget especial para KPIs con valor y delta
- `delta`: Muestra el cambio respecto a un valor base (verde/rojo automático)
- `delta_color="inverse"`: Invierte colores (útil cuando menos es mejor)
- **Orden izquierda → derecha**: Satisfacción (más importante) → Volumen → Eficiencia → Calidad

## 📈 Paso 8: Gráficas de Tendencias (Nivel 2 - Medio)

**PRINCIPIO**: Después de los KPIs, mostramos tendencias y comparaciones. Aquí el usuario puede explorar el "por qué" detrás de los números.

In [ ]:
# Sección de gráficas
st.subheader('📈 Análisis de Tendencias')

# Dos columnas para gráficas lado a lado
col1, col2 = st.columns(2)

with col1:
    # Gráfica 1: Satisfacción por Área
    satisfaccion_por_area = df_filtrado.groupby('area')['satisfaccion'].mean().sort_values(ascending=True)
    
    fig1 = px.bar(
        satisfaccion_por_area,
        orientation='h',
        title='Satisfacción Promedio por Área',
        labels={'value': 'Satisfacción', 'area': 'Área'},
        color=satisfaccion_por_area.values,
        color_continuous_scale='RdYlGn',  # Rojo → Amarillo → Verde
        range_color=[5, 10]
    )
    fig1.update_layout(showlegend=False, height=400)
    st.plotly_chart(fig1, use_container_width=True)

with col2:
    # Gráfica 2: Distribución de Beneficiarios por Programa
    beneficiarios_por_programa = df_filtrado['programa'].value_counts()
    
    fig2 = px.pie(
        values=beneficiarios_por_programa.values,
        names=beneficiarios_por_programa.index,
        title='Distribución de Beneficiarios por Programa',
        hole=0.4  # Gráfica de dona
    )
    fig2.update_layout(height=400)
    st.plotly_chart(fig2, use_container_width=True)

### 🔍 Explicación:
- `st.columns(2)`: Dos gráficas lado a lado (izquierda → derecha)
- `px.bar` con `orientation='h'`: Barras horizontales (mejor para comparar categorías)
- `color_continuous_scale='RdYlGn'`: Escala de color intuitiva (rojo=malo, verde=bueno)
- `px.pie` con `hole=0.4`: Gráfica de dona (más moderna que pie chart tradicional)
- `use_container_width=True`: La gráfica usa todo el ancho de su columna

## 📉 Paso 9: Gráfica de Serie Temporal

In [ ]:
# Gráfica 3: Evolución temporal (ancho completo)
st.markdown('### 📅 Evolución Temporal')

# Preparar datos por mes
evolucion_mensual = df_filtrado.groupby('mes').agg({
    'satisfaccion': 'mean',
    'beneficiario_id': 'count',
    'tiempo_atencion_min': 'mean'
}).reset_index()

evolucion_mensual.columns = ['mes', 'satisfaccion_promedio', 'total_beneficiarios', 'tiempo_promedio']

# Crear gráfica con dos ejes Y
fig3 = go.Figure()

# Línea 1: Satisfacción (eje Y izquierdo)
fig3.add_trace(go.Scatter(
    x=evolucion_mensual['mes'],
    y=evolucion_mensual['satisfaccion_promedio'],
    name='Satisfacción',
    mode='lines+markers',
    line=dict(color='#2ecc71', width=3),
    marker=dict(size=8)
))

# Barras: Total beneficiarios (eje Y derecho)
fig3.add_trace(go.Bar(
    x=evolucion_mensual['mes'],
    y=evolucion_mensual['total_beneficiarios'],
    name='Beneficiarios',
    yaxis='y2',
    marker=dict(color='#3498db', opacity=0.6)
))

# Configurar dos ejes Y
fig3.update_layout(
    title='Satisfacción y Volumen de Beneficiarios por Mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Satisfacción Promedio', range=[0, 10]),
    yaxis2=dict(title='Total Beneficiarios', overlaying='y', side='right'),
    height=400,
    hovermode='x unified'
)

st.plotly_chart(fig3, use_container_width=True)

### 🔍 Explicación:
- `go.Figure()`: API de bajo nivel de Plotly para mayor control
- Dos ejes Y: Satisfacción (izquierda) y Volumen (derecha)
- `hovermode='x unified'`: Al pasar el mouse, muestra todos los valores de ese mes
- Esta gráfica ocupa todo el ancho (no está en columnas)

## 📋 Paso 10: Tabla de Detalle (Nivel 3 - Abajo)

**PRINCIPIO**: Los detalles van al final. Usuarios que quieren profundizar pueden scrollear hacia abajo.

In [ ]:
st.markdown('---')
st.subheader('📋 Detalle de Beneficiarios')

# Selector de columnas a mostrar
columnas_disponibles = df_filtrado.columns.tolist()
columnas_mostrar = st.multiselect(
    'Selecciona las columnas a mostrar:',
    columnas_disponibles,
    default=['beneficiario_id', 'nombre', 'area', 'programa', 'satisfaccion', 'resolucion_exitosa']
)

if columnas_mostrar:
    # Mostrar tabla con capacidad de ordenar
    st.dataframe(
        df_filtrado[columnas_mostrar].sort_values('satisfaccion', ascending=False),
        use_container_width=True,
        height=400
    )
    
    # Botón para descargar datos
    csv = df_filtrado[columnas_mostrar].to_csv(index=False).encode('utf-8')
    st.download_button(
        label="📥 Descargar datos filtrados (CSV)",
        data=csv,
        file_name='datos_filtrados_ong.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ Selecciona al menos una columna para ver la tabla')

### 🔍 Explicación:
- `st.multiselect`: Permite seleccionar múltiples opciones
- `st.dataframe`: Tabla interactiva con scroll y ordenamiento
- `st.download_button`: Permite descargar los datos filtrados
- Tabla ordenada por satisfacción (de mayor a menor) por defecto

## 📊 Paso 11: Análisis Avanzado (Opcional)

In [ ]:
# Sección colapsable para análisis avanzado
with st.expander('🔬 Análisis Avanzado'):
    st.markdown('### Relación entre Tiempo de Atención y Satisfacción')
    
    # Scatter plot
    fig4 = px.scatter(
        df_filtrado,
        x='tiempo_atencion_min',
        y='satisfaccion',
        color='resolucion_exitosa',
        size='satisfaccion',
        hover_data=['nombre', 'area', 'programa'],
        title='¿Más tiempo = Más satisfacción?',
        labels={
            'tiempo_atencion_min': 'Tiempo de Atención (minutos)',
            'satisfaccion': 'Satisfacción',
            'resolucion_exitosa': 'Resolución'
        },
        color_discrete_map={'SI': '#2ecc71', 'NO': '#e74c3c'}
    )
    fig4.update_layout(height=500)
    st.plotly_chart(fig4, use_container_width=True)
    
    # Insight automático
    correlacion = df_filtrado[['tiempo_atencion_min', 'satisfaccion']].corr().iloc[0, 1]
    
    if correlacion > 0.3:
        st.success(f'✅ Correlación positiva moderada ({correlacion:.2f}): Más tiempo podría estar asociado con mayor satisfacción.')
    elif correlacion < -0.3:
        st.error(f'❌ Correlación negativa moderada ({correlacion:.2f}): Más tiempo está asociado con menor satisfacción.')
    else:
        st.info(f'ℹ️ Correlación débil ({correlacion:.2f}): No hay relación clara entre tiempo y satisfacción.')

### 🔍 Explicación:
- `with st.expander()`: Crea sección colapsable para no saturar el dashboard
- `px.scatter`: Gráfica de dispersión para ver relaciones entre variables
- Cálculo automático de correlación con insights dinámicos
- `color_discrete_map`: Colores personalizados (verde=SI, rojo=NO)

## 🎯 Paso 12: Footer con Información

In [ ]:
# Footer
st.markdown('---')
st.markdown(
    """
    <div style='text-align: center; color: #7f8c8d;'>
    <p><strong>Dashboard creado con Streamlit</strong></p>
    <p>CD2001B - Diagnóstico para Líneas de Acción | Tec de Monterrey</p>
    <p>📊 Última actualización: {} registros cargados</p>
    </div>
    """.format(len(df)),
    unsafe_allow_html=True
)

## 🚀 Paso 13: Ejecutar el Dashboard

### Opción A: Desde la Terminal
```bash
# Navega a la carpeta donde guardaste dashboard_ong.py
cd Semana4/notebooks

# Ejecuta el dashboard
streamlit run dashboard_ong.py
```

### Opción B: Desde VS Code
1. Abre la terminal integrada (Ctrl + `)
2. Ejecuta: `streamlit run dashboard_ong.py`

### ¿Qué pasa al ejecutar?
- Se abrirá automáticamente tu navegador en `http://localhost:8501`
- Verás el dashboard interactivo
- Cada vez que cambies un filtro, el dashboard se actualiza automáticamente

## 📦 Paso 14: Código Completo en un Solo Archivo

Para tu comodidad, aquí está todo el código junto. Copia esto en `dashboard_ong.py`:

In [ ]:
# dashboard_ong.py - CÓDIGO COMPLETO

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# ============================================================================
# CONFIGURACIÓN
# ============================================================================
st.set_page_config(
    page_title="Dashboard ONG",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ============================================================================
# CARGAR DATOS
# ============================================================================
@st.cache_data
def cargar_datos():
    df = pd.read_csv('../datos/datos_ong_ejemplo.csv')
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.to_period('M').astype(str)
    return df

df = cargar_datos()

# ============================================================================
# TÍTULO
# ============================================================================
st.title('📊 Dashboard de Atención a Beneficiarios')
st.markdown('### Análisis de Satisfacción y Operaciones')
st.markdown('---')

# ============================================================================
# SIDEBAR - FILTROS
# ============================================================================
st.sidebar.header('🔍 Filtros')

areas_disponibles = ['Todas'] + sorted(df['area'].unique().tolist())
area_seleccionada = st.sidebar.selectbox('Área Geográfica:', areas_disponibles)

programas_disponibles = ['Todos'] + sorted(df['programa'].unique().tolist())
programa_seleccionado = st.sidebar.selectbox('Programa:', programas_disponibles)

satisfaccion_min, satisfaccion_max = st.sidebar.slider(
    'Rango de Satisfacción:',
    min_value=int(df['satisfaccion'].min()),
    max_value=int(df['satisfaccion'].max()),
    value=(int(df['satisfaccion'].min()), int(df['satisfaccion'].max()))
)

st.sidebar.markdown('---')
st.sidebar.markdown('**💡 Tip**: Cambia los filtros para explorar los datos')

# ============================================================================
# APLICAR FILTROS
# ============================================================================
df_filtrado = df.copy()

if area_seleccionada != 'Todas':
    df_filtrado = df_filtrado[df_filtrado['area'] == area_seleccionada]

if programa_seleccionado != 'Todos':
    df_filtrado = df_filtrado[df_filtrado['programa'] == programa_seleccionado]

df_filtrado = df_filtrado[
    (df_filtrado['satisfaccion'] >= satisfaccion_min) &
    (df_filtrado['satisfaccion'] <= satisfaccion_max)
]

st.info(f'📊 Mostrando **{len(df_filtrado)}** registros de **{len(df)}** totales')

# ============================================================================
# KPIs (ARRIBA)
# ============================================================================
satisfaccion_promedio = df_filtrado['satisfaccion'].mean()
total_beneficiarios = len(df_filtrado)
tiempo_promedio = df_filtrado['tiempo_atencion_min'].mean()
tasa_resolucion = (df_filtrado['resolucion_exitosa'] == 'SI').sum() / len(df_filtrado) * 100

col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        label="😊 Satisfacción Promedio",
        value=f"{satisfaccion_promedio:.1f}/10",
        delta=f"{satisfaccion_promedio - df['satisfaccion'].mean():.1f} vs. total"
    )

with col2:
    st.metric(
        label="👥 Total Beneficiarios",
        value=f"{total_beneficiarios:,}"
    )

with col3:
    st.metric(
        label="⏱️ Tiempo Promedio",
        value=f"{tiempo_promedio:.0f} min",
        delta=f"{tiempo_promedio - df['tiempo_atencion_min'].mean():.0f} vs. total",
        delta_color="inverse"
    )

with col4:
    st.metric(
        label="✅ Tasa de Resolución",
        value=f"{tasa_resolucion:.1f}%"
    )

st.markdown('---')

# ============================================================================
# GRÁFICAS (MEDIO)
# ============================================================================
st.subheader('📈 Análisis de Tendencias')

col1, col2 = st.columns(2)

with col1:
    satisfaccion_por_area = df_filtrado.groupby('area')['satisfaccion'].mean().sort_values(ascending=True)
    fig1 = px.bar(
        satisfaccion_por_area,
        orientation='h',
        title='Satisfacción Promedio por Área',
        labels={'value': 'Satisfacción', 'area': 'Área'},
        color=satisfaccion_por_area.values,
        color_continuous_scale='RdYlGn',
        range_color=[5, 10]
    )
    fig1.update_layout(showlegend=False, height=400)
    st.plotly_chart(fig1, use_container_width=True)

with col2:
    beneficiarios_por_programa = df_filtrado['programa'].value_counts()
    fig2 = px.pie(
        values=beneficiarios_por_programa.values,
        names=beneficiarios_por_programa.index,
        title='Distribución de Beneficiarios por Programa',
        hole=0.4
    )
    fig2.update_layout(height=400)
    st.plotly_chart(fig2, use_container_width=True)

# Serie temporal
st.markdown('### 📅 Evolución Temporal')

evolucion_mensual = df_filtrado.groupby('mes').agg({
    'satisfaccion': 'mean',
    'beneficiario_id': 'count',
    'tiempo_atencion_min': 'mean'
}).reset_index()
evolucion_mensual.columns = ['mes', 'satisfaccion_promedio', 'total_beneficiarios', 'tiempo_promedio']

fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=evolucion_mensual['mes'],
    y=evolucion_mensual['satisfaccion_promedio'],
    name='Satisfacción',
    mode='lines+markers',
    line=dict(color='#2ecc71', width=3),
    marker=dict(size=8)
))
fig3.add_trace(go.Bar(
    x=evolucion_mensual['mes'],
    y=evolucion_mensual['total_beneficiarios'],
    name='Beneficiarios',
    yaxis='y2',
    marker=dict(color='#3498db', opacity=0.6)
))
fig3.update_layout(
    title='Satisfacción y Volumen de Beneficiarios por Mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Satisfacción Promedio', range=[0, 10]),
    yaxis2=dict(title='Total Beneficiarios', overlaying='y', side='right'),
    height=400,
    hovermode='x unified'
)
st.plotly_chart(fig3, use_container_width=True)

# ============================================================================
# TABLA DE DETALLE (ABAJO)
# ============================================================================
st.markdown('---')
st.subheader('📋 Detalle de Beneficiarios')

columnas_disponibles = df_filtrado.columns.tolist()
columnas_mostrar = st.multiselect(
    'Selecciona las columnas a mostrar:',
    columnas_disponibles,
    default=['beneficiario_id', 'nombre', 'area', 'programa', 'satisfaccion', 'resolucion_exitosa']
)

if columnas_mostrar:
    st.dataframe(
        df_filtrado[columnas_mostrar].sort_values('satisfaccion', ascending=False),
        use_container_width=True,
        height=400
    )
    csv = df_filtrado[columnas_mostrar].to_csv(index=False).encode('utf-8')
    st.download_button(
        label="📥 Descargar datos filtrados (CSV)",
        data=csv,
        file_name='datos_filtrados_ong.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ Selecciona al menos una columna para ver la tabla')

# ============================================================================
# ANÁLISIS AVANZADO
# ============================================================================
with st.expander('🔬 Análisis Avanzado'):
    st.markdown('### Relación entre Tiempo de Atención y Satisfacción')
    
    fig4 = px.scatter(
        df_filtrado,
        x='tiempo_atencion_min',
        y='satisfaccion',
        color='resolucion_exitosa',
        size='satisfaccion',
        hover_data=['nombre', 'area', 'programa'],
        title='¿Más tiempo = Más satisfacción?',
        labels={
            'tiempo_atencion_min': 'Tiempo de Atención (minutos)',
            'satisfaccion': 'Satisfacción',
            'resolucion_exitosa': 'Resolución'
        },
        color_discrete_map={'SI': '#2ecc71', 'NO': '#e74c3c'}
    )
    fig4.update_layout(height=500)
    st.plotly_chart(fig4, use_container_width=True)
    
    correlacion = df_filtrado[['tiempo_atencion_min', 'satisfaccion']].corr().iloc[0, 1]
    if correlacion > 0.3:
        st.success(f'✅ Correlación positiva moderada ({correlacion:.2f}): Más tiempo podría estar asociado con mayor satisfacción.')
    elif correlacion < -0.3:
        st.error(f'❌ Correlación negativa moderada ({correlacion:.2f}): Más tiempo está asociado con menor satisfacción.')
    else:
        st.info(f'ℹ️ Correlación débil ({correlacion:.2f}): No hay relación clara entre tiempo y satisfacción.')

# ============================================================================
# FOOTER
# ============================================================================
st.markdown('---')
st.markdown(
    f"""
    <div style='text-align: center; color: #7f8c8d;'>
    <p><strong>Dashboard creado con Streamlit</strong></p>
    <p>CD2001B - Diagnóstico para Líneas de Acción | Tec de Monterrey</p>
    <p>📊 Última actualización: {len(df)} registros cargados</p>
    </div>
    """,
    unsafe_allow_html=True
)

## 🎓 Resumen de Principios de Diseño Aplicados

### 1. **Jerarquía Visual (Arriba → Abajo)**
✅ KPIs arriba (lo más importante)  
✅ Gráficas de tendencias en el medio  
✅ Tabla de detalle abajo  

### 2. **Flujo de Lectura (Izquierda → Derecha)**
✅ KPIs ordenados por importancia (Satisfacción → Volumen → Tiempo → Resolución)  
✅ Gráficas en columnas lado a lado  

### 3. **General → Particular**
✅ Empieza con promedios generales (KPIs)  
✅ Sigue con comparaciones (por área, por programa)  
✅ Termina con datos individuales (tabla)  

### 4. **Controles en Sidebar**
✅ Todos los filtros en el sidebar izquierdo  
✅ Área principal dedicada a visualizaciones  

### 5. **Interactividad**
✅ Filtros que actualizan todo en tiempo real  
✅ Gráficas con hover para detalles  
✅ Tabla ordenable y descargable  

### 6. **Performance**
✅ Cache de datos (`@st.cache_data`)  
✅ Carga rápida de visualizaciones  

## 📚 Recursos Adicionales

### Documentación Oficial
- [Streamlit Docs](https://docs.streamlit.io/)
- [Plotly Python Docs](https://plotly.com/python/)
- [Pandas Docs](https://pandas.pydata.org/docs/)

### Tutoriales Recomendados
- [Streamlit Gallery](https://streamlit.io/gallery) - Dashboards de ejemplo
- [Plotly Chart Types](https://plotly.com/python/) - Todos los tipos de gráficas

### Despliegue (Deploy)
- [Streamlit Community Cloud](https://streamlit.io/cloud) - Hosting GRATUITO
- Sube tu repo a GitHub y conecta con Streamlit Cloud
- Tu dashboard estará en `https://tu-app.streamlit.app`

## 🎯 Ejercicio Propuesto

### Nivel 1: Básico
1. Ejecuta el dashboard y familiarízate con los filtros
2. Cambia los colores de las gráficas
3. Agrega un nuevo KPI (por ejemplo, edad promedio)

### Nivel 2: Intermedio
1. Agrega un filtro por género en el sidebar
2. Crea una nueva gráfica de tiempo promedio por programa
3. Modifica la paleta de colores del dashboard

### Nivel 3: Avanzado
1. Integra tus propios datos de la ONG que seleccionaste
2. Agrega un selector de tipo de gráfica (barras vs línea)
3. Implementa un sistema de alertas (ej: si satisfacción < 6, mostrar warning)
4. Agrega autenticación básica con `st.secrets`

## ✅ Checklist de Entregables

Para tu proyecto final, asegúrate de:

- [ ] Dashboard funcional con al menos 3 KPIs
- [ ] Mínimo 3 tipos diferentes de gráficas
- [ ] Filtros interactivos en sidebar
- [ ] Tabla de detalle con capacidad de descarga
- [ ] Aplicación correcta de principios de diseño (jerarquía, flujo)
- [ ] Comentarios explicando decisiones de diseño
- [ ] README con instrucciones de ejecución
- [ ] (Opcional) Dashboard desplegado en Streamlit Cloud

## 🎬 Conclusión

Has aprendido a:

1. ✅ Crear dashboards interactivos con Python
2. ✅ Aplicar principios de diseño de dashboards (arriba→abajo, izquierda→derecha)
3. ✅ Implementar filtros dinámicos
4. ✅ Usar Plotly para visualizaciones avanzadas
5. ✅ Optimizar performance con caching
6. ✅ Estructurar código de forma profesional

**Streamlit vs Looker Studio:**

| Característica | Streamlit | Looker Studio |
|----------------|-----------|---------------|
| Código | Python | No-code |
| Flexibilidad | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| Curva aprendizaje | Media (Python) | Baja |
| Costo | Gratis | Gratis |
| Uso ideal | Técnico, customización | Negocios, rapidez |

**Ambas herramientas son complementarias, no excluyentes.**

---

**¡Felicidades! Ahora tienes una alternativa profesional para crear dashboards con código.** 🎉